# 🚀 Training Tự Động trên Google Colab

Notebook này sẽ tự động:
1. ✅ Cài đặt dependencies
2. ✅ Kiểm tra GPU
3. ✅ Mount Google Drive
4. ✅ Download code từ Google Drive
5. ✅ Download/Setup dữ liệu
6. ✅ Chạy training tự động
7. ✅ Lưu kết quả về Google Drive

**Lưu ý**: Chọn GPU runtime trước khi chạy!

In [13]:
# Cài đặt dependencies
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install -q albumentations tqdm tensorboard onnx onnxruntime
%pip install -q pandas numpy Pillow opencv-python
%pip install -q kagglehub

print("✅ Đã cài đặt xong các thư viện!")

✅ Đã cài đặt xong các thư viện!


In [15]:
import torch
import torch.cuda as cuda

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {cuda.is_is_available()}")
if cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}") # Corrected line
    print(f"GPU device: {cuda.get_device_name(0)}")
    print(f"GPU memory: {cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  Không có GPU! Vui lòng chọn GPU runtime:")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU")

PyTorch version: 2.9.0+cu126


AttributeError: module 'torch.cuda' has no attribute 'is_is_available'

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Tạo thư mục lưu kết quả
DRIVE_RESULTS_DIR = '/content/drive/MyDrive/age_gender_emotion_training'
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)
print(f"✅ Đã mount Google Drive")
print(f"📁 Kết quả sẽ lưu tại: {DRIVE_RESULTS_DIR}")

In [ ]:
import zipfile
import os
from pathlib import Path

# ============================================================
# OPTION 1: Clone từ GitHub (KHUYẾN NGHỊ - Tự động cập nhật)
# ============================================================
USE_GITHUB = True  # Đổi thành False nếu muốn dùng zip
GITHUB_REPO_URL = "https://github.com/khoiabc2020/age-gender-emotion-detection.git"  # Repository URL
REPO_NAME = "age-gender-emotion-detection"

if USE_GITHUB:
    repo_dir = Path(f'/content/{REPO_NAME}')
    project_dir = repo_dir / 'training_experiments'

    if repo_dir.exists():
        # Pull code mới nhất
        print("🔄 Đang pull code mới nhất từ GitHub...")
        os.system(f"cd {repo_dir} && git pull")
        print("✅ Đã cập nhật code từ GitHub")
    else:
        # Clone repo
        print(f"📥 Đang clone từ GitHub: {GITHUB_REPO_URL}")
        os.system(f"cd /content && git clone {GITHUB_REPO_URL}")

        if repo_dir.exists():
            print(f"✅ Đã clone code từ GitHub")
            print(f"📁 Project directory: {project_dir}")
        else:
            print("❌ Lỗi khi clone. Kiểm tra lại URL repo.")
            USE_GITHUB = False

# ============================================================
# OPTION 2: Download từ Google Drive (file zip)
# ============================================================
if not USE_GITHUB:
    # Đường dẫn file zip trên Drive
    import glob
    zip_files = glob.glob('/content/drive/MyDrive/Colab_Training/training_experiments_*.zip')

    if zip_files:
        # Lấy file mới nhất
        latest_zip = max(zip_files, key=os.path.getctime)
        print(f"📦 Tìm thấy file: {os.path.basename(latest_zip)}")

        # Giải nén
        print("📂 Đang giải nén...")
        with zipfile.ZipFile(latest_zip, 'r') as zip_ref:
            zip_ref.extractall('/content/project')
        print("✅ Đã giải nén code")
    else:
        print("⚠️  Không tìm thấy file zip trên Drive")
        print("   Hãy upload file zip vào: /content/drive/MyDrive/Colab_Training/")

# Kiểm tra cấu trúc
project_dir = Path('/content/project/training_experiments')
if project_dir.exists():
    print(f"\n✅ Code đã sẵn sàng tại: {project_dir}")
else:
    print("\n⚠️  Cần kiểm tra lại cấu trúc thư mục")

In [ ]:
import shutil
from pathlib import Path

# Option 1: Copy từ Google Drive (nếu đã upload)
DRIVE_DATA_DIR = '/content/drive/MyDrive/age_gender_emotion_data'
LOCAL_DATA_DIR = Path('/content/project/training_experiments/data/processed')

if os.path.exists(DRIVE_DATA_DIR):
    print(f"📁 Copy dữ liệu từ Drive...")
    shutil.copytree(DRIVE_DATA_DIR, LOCAL_DATA_DIR, dirs_exist_ok=True)
    print(f"✅ Đã copy dữ liệu")
else:
    print(f"⚠️  Chưa có dữ liệu trên Drive")
    print(f"   Upload dữ liệu vào: {DRIVE_DATA_DIR}")
    print(f"   Hoặc sử dụng cell tiếp theo để download từ Kaggle")

In [ ]:
from pathlib import Path

data_dir = Path('/content/project/training_experiments/data/processed')
train_dir = data_dir / 'train'
val_dir = data_dir / 'val'
test_dir = data_dir / 'test'

print("📊 Kiểm tra dữ liệu:")
print(f"   Train: {train_dir.exists()}")
print(f"   Val: {val_dir.exists()}")
print(f"   Test: {test_dir.exists()}")

if train_dir.exists():
    train_images = list(train_dir.glob('**/*.jpg')) + list(train_dir.glob('**/*.png'))
    val_images = list(val_dir.glob('**/*.jpg')) + list(val_dir.glob('**/*.png')) if val_dir.exists() else []
    test_images = list(test_dir.glob('**/*.jpg')) + list(test_dir.glob('**/*.png')) if test_dir.exists() else []

    print(f"\n📸 Số lượng ảnh:")
    print(f"   Train: {len(train_images)}")
    print(f"   Val: {len(val_images)}")
    print(f"   Test: {len(test_images)}")

    if len(train_images) == 0:
        print("\n⚠️  Chưa có ảnh trong thư mục train!")
else:
    print("\n⚠️  Chưa có dữ liệu processed!")

In [ ]:
import sys
import os
from pathlib import Path

# Thêm project vào Python path
project_dir = Path('/content/project/training_experiments')
sys.path.insert(0, str(project_dir))
os.chdir(project_dir)

# Cấu hình training
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
USE_QAT = True
USE_DISTILLATION = True

print("🚀 Bắt đầu training tự động...")
print("=" * 60)
print(f"⚙️  Cấu hình:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   QAT: {USE_QAT}")
print(f"   Distillation: {USE_DISTILLATION}")
print("=" * 60)

# Build command
cmd = f"python train_week2_lightweight.py --data_dir data/processed --epochs {EPOCHS} --batch_size {BATCH_SIZE} --lr {LEARNING_RATE} --save_dir checkpoints/week2_colab --num_workers 2"

if USE_QAT:
    cmd += " --use_qat"
if USE_DISTILLATION:
    cmd += " --use_distillation"

# Chạy training
os.system(cmd)

print("\n" + "=" * 60)
print("✅ Training hoàn tất!")

In [ ]:
import shutil
from datetime import datetime
from pathlib import Path

# Tạo thư mục lưu kết quả với timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_dir = Path(DRIVE_RESULTS_DIR) / f'training_{timestamp}'
results_dir.mkdir(parents=True, exist_ok=True)

# Copy checkpoints
checkpoint_dir = Path('/content/project/training_experiments/checkpoints/week2_colab')
if checkpoint_dir.exists():
    print(f"📦 Copy checkpoints...")
    shutil.copytree(checkpoint_dir, results_dir / 'checkpoints', dirs_exist_ok=True)
    print(f"✅ Đã copy checkpoints")

# Copy logs
logs_dir = checkpoint_dir / 'logs'
if logs_dir.exists():
    print(f"📊 Copy logs...")
    shutil.copytree(logs_dir, results_dir / 'logs', dirs_exist_ok=True)
    print(f"✅ Đã copy logs")

# Copy ONNX model
onnx_file = checkpoint_dir / 'mobileone_multitask.onnx'
if onnx_file.exists():
    print(f"📄 Copy ONNX model...")
    shutil.copy2(onnx_file, results_dir / 'mobileone_multitask.onnx')
    print(f"✅ Đã copy ONNX model")

print(f"\n✅ Tất cả kết quả đã được lưu tại:")
print(f"   {results_dir}")